# Feature Vectors
The purpose of this notebook is to write out formulas for calculating feature vector updates and to document the thought process for the implementation of feature vectors and the dictionary outside of the final program.

## Statistical Calculations

### Average
Below is the formula for calculating the current average, $\mu_{n}$, for different fields with only the calculated average from one previous time step, $\mu_{n-1}$, and the current value update for the field, $x_{n}$. This can be used on any of the fields present in the track dataset.

$$
\mu_{n} = \frac{(n-1)\mu_{n-1} + x_{n}}{n}
$$

### Standard Deviation
Below is the formula for calculating the current standard deviation, $\sigma_{n}$, with only the previously calculated standard deviation from the last time step, $\sigma_{n-1}$, and the current value update for the field, $x_{n}$. This can be used on any of the fields present in the track dataset.

$$
\sigma_{n} = \sqrt{\frac{(n-2)\sigma_{n-1}^{2} + (x_{n}-\mu_{n})*(x_{n}-\mu_{n-1})}{n-1}}
$$

## Smoothness Calculations

Below are the formulas for calculating both the smoothness $M(1)$ and $M(2)$ metrics. These metrics convey the relative visual smoothness of a signal processed from any of the fields either provided by the kinematic track data or features calculated throughout the script. Added variables $q$ and $c$ are also specified in the equations. $z_n$ is any field or calculated feature at the current time interval.
$$
q = \frac{1}{n}(q_{n-1} * (n-1) + (z_{n} - z_{n-1}))
$$
$$
M(1) = \frac{1}{n}(M(1)_{n-1}*(n-1)+(z_{n}-z_{n-1}-q)^2)^2
$$
$$
c = \frac{1}{n}(c_{n-1}*(n-1)+|z_{n}-z_{n-1}|)
$$
$$
M(2)=\frac{M(1)}{c}
$$